<a href="https://colab.research.google.com/github/sprutxz/English-To-Python/blob/main/EnglishToPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
from timeit import default_timer as timer
import math

import torch
import torch.nn as nn
from torch import Tensor
from torch.nn import Transformer
import math
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import gensim
from gensim.models import Word2Vec
from tokenize import tokenize, untokenize
import io
import re
#from nltk.stem import PorterStemmer

# Setting the device for model
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

%matplotlib inline

In [3]:
# Examining the dataset
with open('english_python_data.txt',"r") as data_file:
  print(data_file.readlines()[:11]) # Printing out the last 5 lines of the data

['# write a python program to add two numbers \n', 'num1 = 1.5\n', 'num2 = 6.3\n', 'sum = num1 + num2\n', "print(f'Sum: {sum}')\n", '\n', '\n', '# write a python function to add two user provided numbers and return the sum\n', 'def add_two_numbers(num1, num2):\n', '    sum = num1 + num2\n', '    return sum\n']


In [3]:
#custom tokenizer for python code
def tokenize_python_code(python_code_str):
    python_tokens = list(tokenize(io.BytesIO(python_code_str.encode('utf-8')).readline))
    tokenized_output = []
    for i in range(0, len(python_tokens)):
        tokenized_output.append((python_tokens[i].type, python_tokens[i].string))
    return tokenized_output

tokenizer = get_tokenizer('spacy',language='en_core_web_sm') #tokenizer function for the english text

In [13]:
# Creating a Regular Expression (Regex) pattern of urls to remove them
url_pattern = re.compile(r"https?://\S+")

src_doc = []
tgt_doc = []
i = 0
# Making a dataset
with open('english_python_data.txt',"r") as data_file:
  data_lines = data_file.readlines()
  dps = [] # List of dictionaries
  dp = None # The current problem and solution
  for line in data_lines:
    if line[0] == "#":
      tab_count = 0
      loop_indent = 0
      dict = {0: 0}
      if dp:
        dp['solution'] = ''.join(dp['solution'])
        dp['solution'] = re.sub(r'\n+', '\n', dp['solution']) #replaces multiple newlines with a single newline
        dp['solution'] = re.sub(r'(\n\t*)*$', '', dp['solution']) #removes any newlines at the end of the solution
        dp['solution'] = re.sub(r'^\n', '', dp['solution']) #removes any newlines at the beginning of the solution
          
        tgt_doc.append(tokenize_python_code(dp['solution']))  
        dps.append(dp)
      dp = {"question": None, "solution": []}
      dp['question'] = line[1:].strip("\n ") # Removing any \n in the question
      dp['question'] = re.sub(r'^\d+ ', "", dp['question']) # If the question starts with numbers, I remove them.
      dp['question'] = url_pattern.sub('',dp['question']) # Replacing any urls
      dp['question'] = dp['question'].lower() # lowercasing the question
      dp['question'] = re.sub(r"([.!?])","",dp['question']) # removing any punctuation
      src_doc.append(tokenizer(dp['question'])) # Splitting the question into words
      i+=1
      print(i)
    else:
      line = re.sub(r'( |\t)*\n( |\t)*', '\n', line) #replaces spaces before a newline with a  just a newline
      if line == '\n':
        continue
      
      space_count = len(line) - len(line.lstrip(' '))
      if loop_indent > space_count:
        tab_count = dict[space_count]
        loop_indent = space_count
      if(len(dp['solution'])!=0):
        if (bool(re.search(r':\n$', dp['solution'][-1]))):
          dict[space_count] = tab_count
          loop_indent = space_count

      
      if tab_count > 0 and bool(re.search(r',\n', dp['solution'][-1])!=True):
        line = re.sub(r'^ *', '\t'*tab_count, line)#replaces the first space with a tab
      
      if re.search(r'^(if|else|elif|for|while|def|class|try|except|finally|with)', line):
        tab_count += 1 
      
      
      dp['solution'].append(line)
      

# converting the data to a table for easier viewing
dataset = pd.DataFrame(dps)
dataset

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 6)

In [73]:
# Looking at the first question and the corresponding solution
print(dataset.loc[0,'question'])
print(dataset.loc[0,'solution'])

write a python program to add two numbers
num1 = 1.5
num2 = 6.3
sum = num1 + num2
print(f'Sum: {sum}')


In [3]:
# Creating dictionaries for the tokenizers and the vocabularies
SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'python'

vocabularies = {}

# Generating special characters and their indicies
UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3 # Tokens for Unknown, Padding, start of sentence, end of sentence
special_symbols = ['<unk>','<pad>','<sos>','<eos>']

# Generating the tokens
# This function returns
def yield_tokens(data_iter):
  for data_sample in data_iter:
    yield tokenizer(data_sample)

# Generating the tokens and making the vocabularies based off of them for SRC_LANGUAGE and TGT_LANGUAGE
vocabularies[SRC_LANGUAGE] = build_vocab_from_iterator(yield_tokens(dataset['question']),min_freq=1,specials=special_symbols,special_first=True)
    
vocabularies[TGT_LANGUAGE] = build_vocab_from_iterator(yield_tokens(dataset['solution']),min_freq=1,specials=special_symbols,special_first=True)

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocabularies[ln].set_default_index(UNK_IDX) # Setting the unknown index to be the default index when a token isn't found

In [189]:
for line in dataset['solution']:
    tgt_doc.append(tokenizer(line)) 

In [8]:
# Viewing how big our vocabularies are
print(f"length of english vocab: {vocabularies[SRC_LANGUAGE].__len__()}")
print(f"length of python vocab: {vocabularies[TGT_LANGUAGE].__len__()}")

length of english vocab: 2417
length of python vocab: 11526


In [18]:
src_emb = Word2Vec(src_doc,vector_size=63,workers=3,min_count=1,window=10,sg=1) # Creating a word2vec model for the source language
src_emb.train(src_doc,total_examples=src_emb.corpus_count,epochs=50) # Training the model

# tgt_emb = Word2Vec(tgt_doc,vector_size=63,workers=3,min_count=1,window=10,sg=1) # Creating a word2vec model for the target language
# tgt_emb.train(tgt_doc,total_examples=tgt_emb.corpus_count,epochs=50) # Training the model

(1724365, 3165900)

In [31]:
tgt_emb = Word2Vec(tgt_doc,vector_size=63,workers=3,min_count=1,window=10,sg=1) # Creating a word2vec model for the target language
tgt_emb.train(tgt_doc,total_examples=tgt_emb.corpus_count,epochs=60) # Training the model

(5826554, 11098020)

In [35]:
tokenized_sample = tokenize_python_code(dps[5]['solution'])
print(tokenized_sample)

[(63, 'utf-8'), (1, 'num'), (54, '='), (2, '337'), (4, '\n'), (1, 'if'), (1, 'num'), (54, '>'), (2, '1'), (54, ':'), (4, '\n'), (5, '\t'), (1, 'for'), (1, 'i'), (1, 'in'), (1, 'range'), (54, '('), (2, '2'), (54, ','), (1, 'num'), (54, '//'), (2, '2'), (54, '+'), (2, '1'), (54, ')'), (54, ':'), (4, '\n'), (5, '\t\t'), (1, 'if'), (54, '('), (1, 'num'), (54, '%'), (1, 'i'), (54, ')'), (54, '=='), (2, '0'), (54, ':'), (4, '\n'), (5, '\t\t\t'), (1, 'print'), (54, '('), (1, 'num'), (54, ','), (3, '"is not a prime number"'), (54, ')'), (4, '\n'), (1, 'print'), (54, '('), (3, 'f"{i} times {num//i} is {num}"'), (54, ')'), (4, '\n'), (1, 'break'), (4, '\n'), (6, ''), (6, ''), (1, 'else'), (54, ':'), (4, '\n'), (5, '\t\t'), (1, 'print'), (54, '('), (3, 'f"{num} is a prime number"'), (54, ')'), (4, '\n'), (6, ''), (6, ''), (1, 'else'), (54, ':'), (4, '\n'), (5, '\t'), (1, 'print'), (54, '('), (3, 'f"{num} is not a prime number"'), (54, ')'), (4, ''), (6, ''), (0, '')]


In [32]:
similar_words = tgt_emb.wv.most_similar('for', topn=5)
print(similar_words)

[('in', 0.9882213473320007), ('i', 0.750758707523346), ('[', 0.6188474893569946), (']', 0.605705976486206), ('print(i', 0.5668125748634338)]


In [29]:
print(f"word2vec src vocab: {src_emb.wv.vectors.shape}")
print(f"word2vec tgt vocab: {tgt_emb.wv.vectors.shape}")

word2vec src vocab: (2413, 128)
word2vec tgt vocab: (11522, 128)


In [9]:
# Assuming vocab is your vocab object and w2v_model is your word2vec model
vocab_words = set(vocabularies[TGT_LANGUAGE].get_itos())  # Get words from vocab
w2v_words = set(tgt_emb.wv.index_to_key)  # Get words from word2vec vocab



diff = w2v_words - vocab_words


print(diff)

set()


In [10]:
# Positional Encoding module -> this class is the positional encoder (see above for details)
class PositionalEncoding(nn.Module):
  def __init__(self,emb_size:int, dropout:float, maxlen:int = 5000):
    super().__init__()
    den = torch.exp(-torch.arange(0,emb_size,2)*math.log(10000) / emb_size)
    pos = torch.arange(0,maxlen).reshape(maxlen,1)
    pos_embedding = torch.zeros((maxlen,emb_size))
    pos_embedding[:,0::2] = torch.sin(pos * den)
    pos_embedding[:,1::2] = torch.cos(pos * den)
    pos_embedding = pos_embedding.unsqueeze(-2)

    self.dropout = nn.Dropout(dropout)

    # Saving the positional encoding in the model state dict, but making sure PyTorch doesn't "train"
    # these parameters because they don't need to be trained
    self.register_buffer('pos_embedding',pos_embedding)

  def forward(self,token_embedding: Tensor):
    return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# Converting the tokens into embeddings
class TokenEmbedding(nn.Module):
  def __init__(self,vocab_size: int, emb_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,emb_size)
    self.embed_size = emb_size

  def forward(self,tokens:Tensor):
    return self.embedding(tokens.long()) * math.sqrt(self.embed_size) # we multiply by square root of embedding size to scale. The Transformer paper mentions this.

# class TokenEmbedding(nn.Module):
#   def __init__(self, vocab_size: int, emb_size: int, word2vec_model_path: str):
#     super().__init__()
#     self.word2vec_model = Word2Vec(document,vector_size=128,workers=3,min_count=2,window=10,sg=1) # Creating a word2vec model for the source language
#     self.word2vec_model.train(document,total_examples=self.word2vec_model.corpus_count,epochs=10) # Training the model
#     self.embedding = nn.Embedding(vocab_size, emb_size)
#     self.embed_size = emb_size

#     # Initialize the embedding weights with the Word2Vec vectors
#     self.embedding.weight.data.copy_(torch.from_numpy(self.word2vec_model.wv.vectors))

#   def forward(self, tokens: Tensor):
#     return self.embedding(tokens.long()) * math.sqrt(self.embed_size)
  
    

# The Actual Model
class Seq2SeqTransformer(nn.Module):
  def __init__(self, num_encoder_layers:int, num_decoder_layers:int, emb_size:int, nhead:int, src_vocab_size:int, tgt_vocab_size:int, dim_feedforward: int=512, dropout:float = 0.1):
    super().__init__()
    self.transformer = nn.Transformer(d_model=emb_size, nhead=nhead, num_encoder_layers=num_encoder_layers, num_decoder_layers=num_decoder_layers,dim_feedforward=dim_feedforward,dropout=dropout,
                                      batch_first=True)
    self.generator = nn.Linear(emb_size,tgt_vocab_size) # A layer to convert the matrix (seq_len, emb_size) to (seq_len, tgt_vocab_size)
    self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
    self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size,emb_size)

    # Getting the positional encodings
    self.positional_encoding = PositionalEncoding(emb_size,dropout=dropout)

  def forward(self, src:Tensor, trg: Tensor, src_mask: Tensor, tgt_mask: Tensor, src_padding_mask: Tensor, tgt_padding_mask: Tensor,
              memory_key_padding_mask: Tensor):

    # Embedding both the input and output
    src_embedding = self.positional_encoding(self.src_tok_emb(src))
    tgt_embedding = self.positional_encoding(self.tgt_tok_emb(trg))

    # Getting the output
    output = self.transformer(src_embedding, tgt_embedding, src_mask, tgt_mask, None, src_padding_mask,tgt_padding_mask,memory_key_padding_mask)

    # Getting the logits
    return self.generator(output)
  

  # Encoding the input
  def encode(self, src: Tensor, src_mask: Tensor):
    embedding = self.positional_encoding(self.src_tok_emb(src))
    encoder_output = self.transformer.encoder(embedding, src_mask)
    return encoder_output

  # Decoding the output
  def decode(self,tgt:Tensor, memory: Tensor, tgt_mask:Tensor):
    tgt_embedding = self.tgt_tok_emb(tgt)
    return self.transformer.decoder(self.positional_encoding(tgt_embedding), memory, tgt_mask)


In [11]:
# Defining the lookahead mask that will prevent the model from looking ahead during training
# Also need to define masks that will mask the padding tokens.
# If we don't mask the padding tokens, the model will end up taking the values of the padding into account
# into prediction

# Generating the lookahead mask
def generate_square_subsequent_mask(sz):
  mask = (torch.triu(torch.ones((sz,sz),device=DEVICE)) == 1).transpose(0,1)
  mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
  return mask

# Creating the other mask
def create_mask(src, tgt):
  src_seq_len = src.shape[1]
  tgt_seq_len = tgt.shape[1]

  tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
  src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

  src_padding_mask = (src == PAD_IDX)
  tgt_padding_mask = (tgt == PAD_IDX)
  return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [12]:
# Splitting the data into training and testing
#training, testing = train_test_split(dataset,test_size=0.2,random_state=42,shuffle=True)

train_size = int(len(dataset)*0.8)
test_size = len(dataset) - train_size
training, testing = random_split(dataset,[train_size,test_size])

# Running the data through a pipeline to get the transformed and prepared dataset
# helper function to club together sequential operations
def sequential_transforms(*transforms):
  def func(txt_input):
    for transform in transforms:
      txt_input = transform(txt_input)
    return txt_input
  return func

# Function to add BOS/EOS and create tensor for input sequence indicies
def tensor_transform(token_ids):
  return torch.cat((torch.tensor([SOS_IDX]),torch.tensor(token_ids),torch.tensor([EOS_IDX])))

text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  text_transform[ln] = sequential_transforms(tokenizer,vocabularies[ln],tensor_transform) # Tokenize, Convert to Indicies, then Add Special Tokens

# function to put all the data samples into batches
def collate_fn(batch):
  src_batch, tgt_batch = [], []

  # Iterating through the questions
  for X in batch.dataset['question'].values:
    src_batch.append(text_transform[SRC_LANGUAGE](X.strip('\n\t')))

  # Iterating through the solutions
  for y in batch.dataset['solution'].values:
    tgt_batch.append(text_transform[TGT_LANGUAGE](y.strip('\n\t')))

  src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
  tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
  return src_batch.T, tgt_batch.T

In [13]:
# Defining the model, loss function, and optimizer
torch.manual_seed(10)

SRC_VOCAB_SIZE = len(vocabularies[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocabularies[TGT_LANGUAGE])
EMB_SIZE = 128
NHEAD = 4
FFN_HID_DIM = 128
BATCH_SIZE = 8
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

# Defining the model
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

# Setting the parameters using the xavier uniform distribution
for p in transformer.parameters():
  if p.dim() > 1:
    nn.init.xavier_uniform_(p)

# Putting the model on GPU
transformer = transformer.to(DEVICE)

# Defining the loss function
loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_IDX) # makes sure that the padding token doesn't contribute to the loss function!

# Defining the optimizer
optimizer = optim.AdamW(transformer.parameters(),lr=0.0001)

In [14]:
def train_epoch(model,optimizer):
  # Setting the model to training mode
  model.train()
  losses = 0

  # Preparing the data
  X,y = collate_fn(training)
  training_dataset = TensorDataset(X,y)
  train_dataloader = DataLoader(training_dataset, batch_size=BATCH_SIZE)

  # Iterating through the data
  for src, tgt in train_dataloader:
    src = src.to(DEVICE)
    tgt = tgt.to(DEVICE)

    tgt_input = tgt[:,:-1] # Getting the sentence except the EOS since EOS is never inputted to decoder

    # Getting the masks
    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
    logits = model(src,tgt_input, src_mask, tgt_mask, src_padding_mask,tgt_padding_mask,src_padding_mask) # memory is the encoder outputs

    optimizer.zero_grad()
    tgt_out = tgt[:,1:]
    loss = loss_fn(logits.reshape(logits.size(0),-1,1827),tgt_out)
    loss.backward() # Back propagation, calculating the gradients

    optimizer.step()
    losses += loss.item()

  return losses / len(list(train_dataloader)) # Getting the average loss per example

# Evaluation Loop
def evaluate(model):
  model.eval()
  losses = 0

  # Preparing the data
  X,y = collate_fn(testing)
  testing_data = TensorDataset(X,y)
  val_dataloader = DataLoader(testing_data, batch_size=BATCH_SIZE)

  # Iterating through the data
  for src, tgt in val_dataloader:
    src = src.to(DEVICE)
    tgt = tgt.to(DEVICE)

    tgt_input = tgt[:,:-1] # Getting the sentence except the EOS since EOS is never inputted to decoder

    # Getting the masks
    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
    logits = model(src,tgt_input, src_mask, tgt_mask, src_padding_mask,tgt_padding_mask,src_padding_mask) # memory is the encoder outputs
    tgt_out = tgt[:,1:]
    loss = loss_fn(logits.reshape(-1,logits.shape[-1]),tgt_out.reshape(-1))
    losses += loss.item()

  return losses / len(list(val_dataloader)) # Getting the average loss per example

In [15]:
# Training the model
NUM_EPOCHS = 5

for epoch in range(1,NUM_EPOCHS+1):
  start_time = timer()
  train_loss = train_epoch(transformer, optimizer)
  end_time = timer()
  val_loss = evaluate(transformer)
  print(f'Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}')
  print(f'Epoch time: {(end_time - start_time):.3f}s')

/common/home/lms548/miniconda3/envs/nlp/lib/python3.11/site-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


RuntimeError: shape '[8, -1, 1827]' is invalid for input of size 78407280

In [ ]:
# A function to generate the output sequence autoregressively using the greedy decoder algorithm
# Usually, we would utilize something like beam search
def greedy_decode(model, src, max_len, start_symbol):
  src = src.to(DEVICE)
  memory = model.encode(src.view(1,-1), None)
  memory = memory.to(DEVICE)
  ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
  for i in range(max_len-1):
      out = model.decode(ys.view(1,-1), memory, None)
      prob = nn.functional.softmax(model.generator(out[:, -1]),dim=1)
      _, next_word = torch.max(prob, dim=1)
      next_word = next_word.item()

      ys = torch.cat([ys,torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
      if next_word == EOS_IDX:
          break
  return ys

# Function for translation
def translate(model: torch.nn.Module, src_sentence: str):
  model.eval()
  src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
  num_tokens = src.shape[0]
  tgt_tokens = greedy_decode(model,  src, max_len=num_tokens + 5, start_symbol=SOS_IDX).flatten()
  return " ".join(vocabularies[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<sos>", "").replace("<eos>", "")

In [ ]:
# Trying to translate from English to Python
print(translate(transformer, "Give me a function to add 3 numbers."))

 isLucky(next_position isLucky(next_position isLucky(next_position y':5874 y':5874 y':5874 add Traverse y':5874 left y':5874 print('do left Traverse y':5874
